In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import  risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import cvxpy as cp

In [3]:
#导入原始数据和无风险利率数据
dbc = pd.read_csv(r'D:\jupyterworkspace\5225\DBC(origin).csv', sep=',')
lqd = pd.read_csv(r'D:\jupyterworkspace\5225\LQD(origin).csv', sep=',')
qqq = pd.read_csv(r'D:\jupyterworkspace\5225\QQQ(origin).csv', sep=',')
vti = pd.read_csv(r'D:\jupyterworkspace\5225\VTI(origin).csv', sep=',')
rf = pd.read_csv(r'C:\Users\Joyce\Desktop\nus\FE5225\^TNX.csv',sep=',').dropna()

In [4]:
def todateindex(data):
    new_data=data.copy()
    new_data.index=pd.to_datetime(new_data["Date"])
    return(new_data)

In [5]:
#将无风险利率数据和etf数据对齐
dbc_date=todateindex(dbc)
lqd_date=todateindex(lqd)
qqq_date=todateindex(qqq)
vti_date=todateindex(vti)
rf_date =todateindex(rf)
close_all = pd.concat((dbc_date['Close'],vti_date["Close"],qqq_date["Close"],lqd_date["Close"]),axis=1,keys=["dbc","vti","qqq","lqd"])
close_all=close_all.join(rf_date["Close"])
close_all.rename(columns={"Close":"risk_free"},inplace=True)
close_all.index=np.arange(0,len(close_all))

In [6]:
close_all.rename(columns={"Close":"risk_free"},inplace=True)
close_all[close_all["risk_free"].isna()]

,dbc,vti,qqq,lqd,risk_free
1452,15.35,111.139999,119.220001,122.110001,NaN
1476,14.41,111.410004,115.800003,118.080002,NaN


In [7]:
close_all.index=np.arange(0,len(close_all))

In [8]:
#fill in the missing value in risk free rate
index_missing=close_all[close_all["risk_free"].isna()].index
for i in index_missing:
    close_all.loc[i,"risk_free"]=(close_all.loc[i+1,"risk_free"]+close_all.loc[i-1,"risk_free"])/2

In [9]:
#从这里导入预测数据
prediction_all=pd.read_csv("svr_10_1.csv",header=0,index_col=0)
prediction_all.head()

,qqq,vti,lqd,dbc
11,55.840059,65.929877,108.299595,27.642000
12,56.020168,65.780406,108.460122,27.319999
13,55.970105,65.630207,108.810142,27.220348
14,56.179752,65.679565,108.999921,27.540001
15,56.259965,65.939996,108.950003,27.770000


In [10]:
prediction_all=pd.concat((prediction_all["dbc"],prediction_all["vti"],prediction_all["qqq"],prediction_all["lqd"]),axis=1)

In [11]:
close_4=close_all.drop(columns="risk_free")
close_4.head(5)

,dbc,vti,qqq,lqd
0,27.670000,65.599998,55.310001,108.860001
1,27.299999,65.470001,55.270000,109.000000
2,27.590000,65.830002,55.740002,108.199997
3,27.219999,65.680000,55.919998,108.360001
4,27.120001,65.529999,55.869999,108.910004


In [12]:
def input_predicted(beginpoint,rebalance_len,prediction_data,real_data):
    input_index=np.arange(beginpoint,beginpoint+rebalance_len)
    mean_input=prediction_data.loc[input_index,:]#input of S
    S_input=prediction_data.loc[input_index,:]
    if rebalance_len==1:
        S_sub_index=np.arange(beginpoint-10,beginpoint)#keep the length of correlation input is always 10(a week)
        S_sub=real_data.loc[S_sub_index,:]
        S_input=pd.concat((S_sub,mean_input),axis=0)#combine to get the volatility matrix for rebalancing frequency is 1.
        mean_sub_index=np.arange(beginpoint-1,beginpoint)
        mean_sub=real_data.loc[mean_sub_index,:]
        mean_input=pd.concat((mean_sub,mean_input),axis=0)
    return(mean_input,S_input)

In [13]:
def get_rf(beginpoint,rebalance_len,rf):
    index=np.arange(beginpoint,beginpoint+rebalance_len)
    rf_return=rf[index]
    if rebalance_len==1:
        index=np.arange(beginpoint-1,beginpoint+1)
        rf_return=rf[index]
    return(rf_return)

In [14]:
def portfolio_price(real_data,prediction,rf_input,begin_day,rebalance_len,start_money,transaction_cost):
    i=begin_day
    return_amount=0
    transaction_times=0
    moneyamount=start_money
    return_series=[0]
    moneyamount_series=[start_money]
    flag=0
    while(i+rebalance_len)<=len(real_data):
        #use the prediction to get the wight of our portfolio
        rf=get_rf(i,rebalance_len,rf_input)
        mean,S_input=input_predicted(i,rebalance_len,prediction,real_data)
        S=risk_models.sample_cov(S_input)#suppose the varaince is consistent within the prediction and past
        input_pre=pd.concat((mean,rf),axis=1)
        mean=expected_returns.mean_historical_return(input_pre,frequency=len(input_pre)-1)
        input_mean_index=np.arange(0,S.shape[0])
        mean_input=mean.iloc[input_mean_index]
        if sum((mean_input-mean[-1])>1e-8)==0:
            if flag==0:
                cost=0
            else:
                if rebalance_len==1:
                    latest_prices = real_data.loc[i-1,:]
                else:
                    latest_prices = real_data.loc[i,:]
                cost=0
                for c in  portfolio_df["Ticker"]:
                    cost+=-latest_prices[c]*prev_w.loc[c,"Number of stocks to buy"]*transaction_cost
            moneyamount_series.append(moneyamount+cost)
            return_series.append(cost)
            flag=0
            i+=rebalance_len
            print("sell all the asset and now is {}/2768".format(i))
            continue
        else:
            
            ef = EfficientFrontier(mean_input,S,solver=cp.GUROBI)
            weights = ef.max_sharpe(risk_free_rate=mean[-1]) #for maximizing the Sharpe ratio #Optimization
            cleaned_weights = ef.clean_weights() #to clean the raw weight
            print(cleaned_weights)
        
            # Get discrete allocation of each share per stock
            if rebalance_len==1:
                latest_prices = real_data.loc[i-1,:]
            else:
                latest_prices = real_data.loc[i,:]
            #use the prediction to get the discrete_allocation of each etf
            discrete_allocation = DiscreteAllocation(cleaned_weights, latest_prices , total_portfolio_value = int(moneyamount))
            allocation , leftover = discrete_allocation.lp_portfolio()
            discrete_allocation_list = []
            for symbol in ["dbc","vti","lqd","qqq"]:
                if symbol in allocation:
                    discrete_allocation_list.append(allocation.get(symbol))
                else:
                    discrete_allocation_list.append(0)
            portfolio_df = pd.DataFrame(columns =['Ticker' , 'Number of stocks to buy'])
            portfolio_df['Ticker'] = ["dbc","vti","lqd","qqq"]
            portfolio_df['Number of stocks to buy'] = discrete_allocation_list
            portfolio_df.index=["dbc","vti","lqd","qqq"]
            print(portfolio_df)
            #calculate the amount at the beginning of investment
            port_amount=0
            for c in  portfolio_df["Ticker"]:
                #we used to rebalance at the end of last day,so we calculate the value at begin
                port_amount+=latest_prices[c]*portfolio_df.loc[c,"Number of stocks to buy"]
            print("at the beginning the port_amount is {}".format(port_amount))
            #calculate the amount at the end of investment
            if rebalance_len==1:
                future_price = real_data.loc[i,:]
            else:
                future_price = real_data.loc[i+rebalance_len-1,:]
            
            port_amount_future=0
            for c in  portfolio_df["Ticker"]:
                #we need to calculate the return we have got in ith day
                port_amount_future+=future_price[c]*portfolio_df.loc[c,"Number of stocks to buy"]
            if flag==0:
                cost=0
                for c in  portfolio_df["Ticker"]:
                    cost+=-latest_prices[c]*portfolio_df.loc[c,"Number of stocks to buy"]*transaction_cost
            else:
                for c in  portfolio_df["Ticker"]:
                    cost+=-latest_prices[c]*abs((portfolio_df.loc[c,"Number of stocks to buy"]-prev_w.loc[c,"Number of stocks to buy"]))*transaction_cost
            print(" at last the port_amount is{}".format(port_amount_future+cost))
            transaction_times+=1
            return_now=port_amount_future-port_amount+cost
            return_series.append(return_now)
            moneyamount=moneyamount+return_now
            moneyamount_series.append(moneyamount)
            return_amount+=return_now
            print(" this transaction return is{}".format(moneyamount_series[-1]/moneyamount_series[-2]))
            print("now is the end of {}/2768".format(i))
            i+=rebalance_len
            prev_w = portfolio_df
            flag=1
    return(moneyamount_series,return_series,transaction_times)

In [15]:
def portfolio_return(real_data,prediction,rf_input,begin_day,rebalance_len):
    i=begin_day
    acc_return=[1]
    while(i+rebalance_len)<=len(real_data):
        #use the prediction to get the wight of our portfolio
        rf=get_rf(i,rebalance_len,rf_input)
        mean,S_input=input_predicted(i,rebalance_len,prediction,real_data)

        S=risk_models.sample_cov(S_input)#suppose the varaince is consistent within the prediction and past

        input_pre=pd.concat((mean,rf),axis=1)
        
        mean=expected_returns.mean_historical_return(input_pre,frequency=len(input_pre-1))
        
        input_mean_index=np.arange(0,S.shape[0])
        mean_input=mean.iloc[input_mean_index]
        real_return_input,S_real=input_predicted(i,rebalance_len,real_data,real_data)
        real_return=expected_returns.mean_historical_return(real_return_input,frequency=len(real_return_input)-1)
        
        if sum((mean_input-mean[-1])>1e-8)==0:
            print("sell all the asset and now is {}/2768".format(i))
            acc_return.append(acc_return[-1])
            i+=rebalance_len 
            continue
        else:
            ef = EfficientFrontier(mean_input,S,solver=cp.GUROBI)
            weights = ef.max_sharpe(risk_free_rate=mean[-1]) #for maximizing the Sharpe ratio #Optimization
            cleaned_weights = ef.clean_weights() #to clean the raw weight
            allocation_list = []
            allocation=[]
            for symbol in cleaned_weights:
                allocation_list.append(cleaned_weights.get(symbol))
                allocation.append(symbol)
            
            portfolio_df = pd.DataFrame(columns =['Ticker' , 'weight of stocks to buy'])
            portfolio_df['Ticker'] = allocation
            portfolio_df['weight of stocks to buy'] = allocation_list
            portfolio_df.index=allocation
            print(portfolio_df)
            #calculate the amount at the beginning of investment
            per_return=0
            for c in  allocation:
                per_return+=portfolio_df.loc[c,"weight of stocks to buy"]* real_return[c]
            per_return=per_return+1
            print(per_return)
            acc_return_now=acc_return[-1]*(per_return)
            acc_return.append(acc_return_now)
            print("now is {}/2768".format(i))
            i+=rebalance_len
    return(acc_return)

In [16]:
acc_return=portfolio_return(close_4,prediction_all,close_all["risk_free"],1762,20)

sell all the asset and now is 1762/2768
sell all the asset and now is 1782/2768
Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-17
    Ticker  weight of stocks to buy
dbc    dbc                  0.39443
vti    vti                  0.00000
qqq    qqq                  0.00000
lqd    lqd                  0.60557
1.0142398047554353
now is 1802/2768
sell all the asset and now is 1822/2768
    Ticker  weight of stocks to buy
dbc    dbc                      1.0
vti    vti                      0.0
qqq    qqq                      0.0
lqd    lqd                      0.0
1.0261959582116145
now is 1842/2768
    Ticker  weight of stocks to buy
dbc    dbc                      0.0
vti    vti                      0.0
qqq    qqq                      1.0
lqd    lqd                      0.0
1.0142036692063456
now is 1862/2768
sell all the asset and now is 1882/2768
    Ticker  weight of stocks to buy
dbc    dbc                  0.08891
vti    vti                  0

KeyboardInterrupt: 

In [511]:
acc_return[-1]

1.7563581076936439

In [692]:
moneyamount_series,return_series,transaction_times=portfolio_price(close_4,prediction_all,close_all["risk_free"],1762,1,1000000,0.0001)

OrderedDict([('dbc', 0.56196), ('vti', 0.0), ('qqq', 0.20869), ('lqd', 0.22935)])
    Ticker  Number of stocks to buy
dbc    dbc                    33670
vti    vti                        0
lqd    lqd                     1899
qqq    qqq                     1317
at the beginning the port_amount is 999987.920255
 at last the port_amount is1005700.8754599744
 this transaction return is1.0057129552049744
now is the end of 1762/2768
OrderedDict([('dbc', 0.65491), ('vti', 0.0), ('qqq', 0.34509), ('lqd', 0.0)])
    Ticker  Number of stocks to buy
dbc    dbc                    39226
vti    vti                        0
lqd    lqd                        0
qqq    qqq                     2169
at the beginning the port_amount is 1005709.6470570001
 at last the port_amount is1005386.4715863143
 this transaction return is0.9996786603284632
now is the end of 1763/2768
sell all the asset and now is 1765/2768
OrderedDict([('dbc', 0.0), ('vti', 0.92869), ('qqq', 0.0), ('lqd', 0.07131)])
    Ticker  Numbe

    Ticker  Number of stocks to buy
dbc    dbc                    22131
vti    vti                        0
lqd    lqd                     5326
qqq    qqq                        0
at the beginning the port_amount is 987906.870652
 at last the port_amount is991979.8686609349
 this transaction return is1.004122815798558
now is the end of 1796/2768
OrderedDict([('dbc', 0.1204), ('vti', 0.01244), ('qqq', 0.08006), ('lqd', 0.78711)])
    Ticker  Number of stocks to buy
dbc    dbc                     7131
vti    vti                       89
lqd    lqd                     6692
qqq    qqq                      482
at the beginning the port_amount is 991978.6686850002
 at last the port_amount is998043.428876001
 this transaction return is1.0061137338902444
now is the end of 1797/2768
OrderedDict([('dbc', 0.40784), ('vti', 0.0), ('qqq', 0.25535), ('lqd', 0.33681)])
    Ticker  Number of stocks to buy
dbc    dbc                    24146
vti    vti                        0
lqd    lqd               

    Ticker  Number of stocks to buy
dbc    dbc                     7943
vti    vti                        0
lqd    lqd                     7421
qqq    qqq                        0
at the beginning the port_amount is 1006023.322579
 at last the port_amount is1001111.3470610984
 this transaction return is0.9951175594568684
now is the end of 1822/2768
sell all the asset and now is 1824/2768
OrderedDict([('dbc', 0.0), ('vti', 0.0), ('qqq', 1.0), ('lqd', 0.0)])
    Ticker  Number of stocks to buy
dbc    dbc                        0
vti    vti                        0
lqd    lqd                        0
qqq    qqq                     6366
at the beginning the port_amount is 1001117.12817
 at last the port_amount is1016804.760117183
 this transaction return is1.0157713978336298
now is the end of 1824/2768
sell all the asset and now is 1826/2768
OrderedDict([('dbc', 0.0142), ('vti', 0.0), ('qqq', 0.0), ('lqd', 0.9858)])
    Ticker  Number of stocks to buy
dbc    dbc                      856
vt

OrderedDict([('dbc', 0.0), ('vti', 1.0), ('qqq', 0.0), ('lqd', 0.0)])
    Ticker  Number of stocks to buy
dbc    dbc                        0
vti    vti                     7463
lqd    lqd                        0
qqq    qqq                        0
at the beginning the port_amount is 1048327.617463
 at last the port_amount is1048521.2599232538
 this transaction return is1.00018470821759
now is the end of 1855/2768
OrderedDict([('dbc', 0.39606), ('vti', 0.0), ('qqq', 0.0), ('lqd', 0.60394)])
    Ticker  Number of stocks to buy
dbc    dbc                    22894
vti    vti                        0
lqd    lqd                     5564
qqq    qqq                        0
at the beginning the port_amount is 1048535.965978
 at last the port_amount is1051065.4654683874
 this transaction return is1.002412348295604
now is the end of 1856/2768
OrderedDict([('dbc', 0.0), ('vti', 0.0), ('qqq', 0.0), ('lqd', 1.0)])
    Ticker  Number of stocks to buy
dbc    dbc                        0
vti    vti 

OrderedDict([('dbc', 0.0), ('vti', 0.0), ('qqq', 1.0), ('lqd', 0.0)])
    Ticker  Number of stocks to buy
dbc    dbc                        0
vti    vti                        0
lqd    lqd                        0
qqq    qqq                     6059
at the beginning the port_amount is 1040027.313646
 at last the port_amount is1046789.0548051851
 this transaction return is1.0065009476505802
now is the end of 1886/2768
OrderedDict([('dbc', 0.02939), ('vti', 0.25584), ('qqq', 0.0), ('lqd', 0.71478)])
    Ticker  Number of stocks to buy
dbc    dbc                     1774
vti    vti                     1903
lqd    lqd                     6550
qqq    qqq                        0
at the beginning the port_amount is 1046865.242738
 at last the port_amount is1048480.1761680933
 this transaction return is1.0015426186004406
now is the end of 1887/2768
OrderedDict([('dbc', 0.0), ('vti', 0.0), ('qqq', 1.0), ('lqd', 0.0)])
    Ticker  Number of stocks to buy
dbc    dbc                        0
vti 

...

OrderedDict([('dbc', 0.45141), ('vti', 0.22235), ('qqq', 0.30872), ('lqd', 0.01752)])
    Ticker  Number of stocks to buy
dbc    dbc                    44446
vti    vti                     1772
lqd    lqd                      230
qqq    qqq                     1566
at the beginning the port_amount is 1719054.6553759999
 at last the port_amount is1710230.3398784623
 this transaction return is0.9948668153983989
now is the end of 2590/2768
sell all the asset and now is 2592/2768
OrderedDict([('dbc', 0.04876), ('vti', 0.0483), ('qqq', 0.0), ('lqd', 0.90295)])
    Ticker  Number of stocks to buy
dbc    dbc                     4799
vti    vti                      382
lqd    lqd                    11750
qqq    qqq                        0
at the beginning the port_amount is 1710239.663695
 at last the port_amount is1711622.4729156306
 this transaction return is1.000908642996219
now is the end of 2592/2768
sell all the asset and now is 2594/2768
OrderedDict([('dbc', 1.0), ('vti', 0.0), ('qqq',

    Ticker  Number of stocks to buy
dbc    dbc                    12697
vti    vti                        0
lqd    lqd                    11432
qqq    qqq                        0
at the beginning the port_amount is 1743319.261432
 at last the port_amount is1746815.2707468567
 this transaction return is1.0021055734917075
now is the end of 2620/2768
sell all the asset and now is 2622/2768
OrderedDict([('dbc', 0.09633), ('vti', 0.05287), ('qqq', 0.0), ('lqd', 0.8508)])
    Ticker  Number of stocks to buy
dbc    dbc                     8914
vti    vti                      425
lqd    lqd                    11331
qqq    qqq                        0
at the beginning the port_amount is 1746818.6942379999
 at last the port_amount is1759856.9323815766
 this transaction return is1.0075646890906205
now is the end of 2622/2768
sell all the asset and now is 2624/2768
OrderedDict([('dbc', 0.0826), ('vti', 0.28262), ('qqq', 0.0), ('lqd', 0.63478)])
    Ticker  Number of stocks to buy
dbc    dbc      

now is the end of 2650/2768
sell all the asset and now is 2652/2768
OrderedDict([('dbc', 0.16833), ('vti', 0.06707), ('qqq', 0.04559), ('lqd', 0.71901)])
    Ticker  Number of stocks to buy
dbc    dbc                    15834
vti    vti                      540
lqd    lqd                     9532
qqq    qqq                      228
at the beginning the port_amount is 1789320.1303599998
 at last the port_amount is1783978.9678549638
 this transaction return is0.9971147094227433
now is the end of 2652/2768
sell all the asset and now is 2654/2768
sell all the asset and now is 2655/2768
OrderedDict([('dbc', 0.16743), ('vti', 0.0), ('qqq', 0.0), ('lqd', 0.83257)])
    Ticker  Number of stocks to buy
dbc    dbc                    15804
vti    vti                        0
lqd    lqd                    10980
qqq    qqq                        0
at the beginning the port_amount is 1783960.2439199998
 at last the port_amount is1792301.2101316082
 this transaction return is1.0046754672170393
now is

    Ticker  Number of stocks to buy
dbc    dbc                     4897
vti    vti                        0
lqd    lqd                    10769
qqq    qqq                      756
at the beginning the port_amount is 1836230.634949
 at last the port_amount is1840138.781722505
 this transaction return is1.002228469459542
now is the end of 2691/2768
OrderedDict([('dbc', 0.11933), ('vti', 0.0), ('qqq', 0.55428), ('lqd', 0.32639)])
    Ticker  Number of stocks to buy
dbc    dbc                    11256
vti    vti                        0
lqd    lqd                     4430
qqq    qqq                     2710
at the beginning the port_amount is 1840166.8513500001
 at last the port_amount is1838615.383754242
 this transaction return is0.9991569213789241
now is the end of 2692/2768
sell all the asset and now is 2694/2768
sell all the asset and now is 2695/2768
sell all the asset and now is 2696/2768
OrderedDict([('dbc', 0.0), ('vti', 0.30357), ('qqq', 0.0), ('lqd', 0.69643)])
    Ticker  Numbe

now is the end of 2732/2768
sell all the asset and now is 2734/2768
sell all the asset and now is 2735/2768
sell all the asset and now is 2736/2768
sell all the asset and now is 2737/2768
sell all the asset and now is 2738/2768
OrderedDict([('dbc', 0.13837), ('vti', 0.86163), ('qqq', 0.0), ('lqd', 0.0)])
    Ticker  Number of stocks to buy
dbc    dbc                    12540
vti    vti                     6854
lqd    lqd                        0
qqq    qqq                        0
at the beginning the port_amount is 1931309.9994820002
 at last the port_amount is1921880.293562052
 this transaction return is0.9951174749880054
now is the end of 2738/2768
OrderedDict([('dbc', 0.0), ('vti', 0.0), ('qqq', 0.27394), ('lqd', 0.72606)])
    Ticker  Number of stocks to buy
dbc    dbc                        0
vti    vti                        0
lqd    lqd                    10549
qqq    qqq                     1324
at the beginning the port_amount is 1921883.836071
 at last the port_amount is1929

In [687]:
#acc_return=pd.DataFrame(acc_return,columns=["acc_return"])
moneyamount_series=pd.DataFrame(moneyamount_series,columns=["moneyamount_series"])
return_series=pd.DataFrame(return_series,columns=["return_series"])

In [688]:
df=pd.concat((acc_return,moneyamount_series,return_series),axis=1)
df

,acc_return,moneyamount_series,return_series
0,1.000000,1.000000e+06,0.000000
1,1.000000,1.003813e+06,3812.978156
2,1.000000,1.001179e+06,-2634.290623
3,1.009806,9.991714e+05,-2007.266831
4,1.009806,1.001290e+06,111.553295
...,...,...,...
1002,NaN,3.004887e+05,-603.365973
1003,NaN,3.015202e+05,428.147258
1004,NaN,2.997685e+05,-1751.692301
1005,NaN,2.991667e+05,-601.886851


In [689]:
df.to_csv("arima_acccost_10.csv",sep=",")

In [638]:
portfolio_df = pd.DataFrame(columns =['Ticker' , 'Number of stocks to buy'])
portfolio_df['Ticker'] =["dbc","qqq","lqd","vti"]   
portfolio_df['Number of stocks to buy'] = [0,0,0,0]

In [639]:
portfolio_df

,Ticker,Number of stocks to buy
0,dbc,0
1,qqq,0
2,lqd,0
3,vti,0


In [ ]:
real_value_index=np.arange(21,len(dbc))
real_value_qqq=qqq.loc[real_value_index,"Close"]
x=np.arange(0,len(real_value_qqq))
plt.plot(x,prediction_qqq,label="prediction_qqq",linewidth=1)
plt.plot(x,real_value_qqq,label="real",linewidth=0.2)
#plt.plot(x,Y_bar_qqq,label="ma")
plt.legend()

In [ ]:
def judge_stationarity(data_sanya_one):
    dftest = ts.adfuller(data_sanya_one)
    print(dftest)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    stationarity = 1
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value 
        if dftest[0] > value:
                stationarity = 0
    print(dfoutput)
    print("stationary or not: %d" %(stationarity))
    return stationarity
stationarity1 = judge_stationarity(qqq_data['day_return_close'].dropna())
qqq_diff = qqq['Close'].diff()
qqq_diff = qqq_diff.dropna()
stationarity1 = judge_stationarity(qqq_diff)
dbc_diff = dbc['Close'].diff()
dbc_diff = dbc_diff.dropna()
vti_diff = vti['Close'].diff()
vti_diff = vti_diff.dropna()
lqd_diff = lqd['Close'].diff()
lqd_diff = lqd_diff.dropna()

In [ ]:
def draw_acf_pacf(ts,lags):
    f = plt.figure(facecolor='white')
    plt.rcParams['axes.unicode_minus']=False
    ax1 = f.add_subplot(211)
    plot_acf(ts,ax=ax1,lags=lags)  #lags 表示滞后的阶数，值为30，显示30阶的图像
    ax2 = f.add_subplot(212)
    plot_pacf(ts,ax=ax2,lags=lags)
    plt.rcParams['axes.unicode_minus']=False
    plt.subplots_adjust(hspace=0.5)
    plt.show()
draw_acf_pacf(qqq['Close'].dropna(),30)

In [ ]:
def prediction1(moving,periods,data):
    data_close_shifted=data["Close"].shift(periods=moving,axis=0)
    data_open_shifted=dbc["Open"].shift(periods=moving,axis=0)
    data_high_shifted=dbc["High"].shift(periods=moving,axis=0)
    data_low_shifted=dbc["Low"].shift(periods=moving,axis=0)
    data["Volume"]=pd.DataFrame(preprocessing.scale(dbc["Volume"],axis=0))
    data_volume_shifted=dbc["Volume"].shift(periods=moving,axis=0)
    #dbc.index=pd.to_datetime(dbc['Date'])
    data_moving=pd.concat([data_open_shifted,data_high_shifted,data_low_shifted,data_volume_shifted,data_close_shifted],axis=1)
    ###come to prediction
    prediction=[]
    rmse=[]
    numer=[]
    Y_bar=[]
    i=moving
    while (i+periods)<len(dbc):
        input_index=np.arange(i,i+periods)
        x=data_moving.loc[input_index,["Volume","Close"]]#x is the past price and past volume
        y=data.loc[input_index,"Close"]#y is the present price
        y_bar=np.mean(data_moving.loc[input_index,"Close"])
        model = LinearRegression()
        model.fit(x,y)
        out=np.sum(model.coef_*data_moving.loc[i+periods,["Volume","Close"]]+model.intercept_)
        obs=data.loc[i+periods,"Close"]#
        prediction.append(out)
        rmse.append((out-obs)**2)
        numer.append((obs-y_bar)**2)
        Y_bar.append(y_bar)
        i+=1
    RMSE_linear=np.sqrt(np.sum(rmse)/len(prediction))
    RMSE_ma=np.sqrt(np.sum(numer)/len(prediction))
    return(prediction, Y_bar,RMSE_linear,RMSE_ma)
